In [10]:
import pandas as pd
pd.options.display.max_columns = None # Muestra todas las columnas de los dataframes
import json
import warnings
warnings.filterwarnings("ignore") # No muestra advertencias 
import re
import ast
import tools
import pyarrow

In [11]:
# Necesito relacionar item_id con el developer. Leo el dataframe steam_output
df1 = pd.read_parquet("Tablas/steam_games_output_limpio.parquet")
df1

,genres,title,id,developer,release_year
0,Action,Lost Summoner Kitty,761140,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,761140,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,761140,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,761140,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,761140,Kotoshiro,2018
...,...,...,...,...,...
68605,Indie,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68606,Racing,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68607,Simulation,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68608,Casual,EXIT 2 - Directions,658870,"xropi,stev3ns",2017


In [12]:
# Me quedo con las columnas 'id' y 'developer'
df2 = df1.drop(["genres", "release_year", "title"], axis=1)
# Elimino duplicados
df2.drop_duplicates(subset='id', inplace=True)
df2.reset_index(drop=True, inplace=True)

In [13]:
df2['id'] = df2['id'].astype(int)

In [14]:
# Leo el dataframe auxiliar f4aux y renombro la columna 'id'
df3 = pd.read_parquet("f4aux.parquet")
df2.rename(columns={"id":"item_id"}, inplace=True)
df3


,item_id,recommend,posted_year,sentiment
0,1250,1,2011,1
1,251610,1,2014,1
2,250320,1,2013,2
3,211420,1,2014,1
4,249130,1,2014,0
...,...,...,...,...
48221,332310,1,2015,1
48222,440,1,2014,1
48223,304930,1,2014,1
48224,265630,1,2015,2


In [15]:
# Uno df2 con df3
df4 = pd.merge (df2, df3, on="item_id", how="inner")
df4


,item_id,developer,recommend,posted_year,sentiment
0,70,Valve,1,2015,1
1,70,Valve,1,2014,1
2,70,Valve,1,2015,1
3,70,Valve,1,2014,2
4,70,Valve,1,2011,2
...,...,...,...,...,...
39572,80,Valve,0,2014,1
39573,80,Valve,1,2013,2
39574,80,Valve,1,2011,2
39575,80,Valve,1,2014,2


In [16]:
# Este código se ejecutó una única vez para guardar 'f5aux'
# Lo guardo en parquet para usarlo en la API
# df4.to_parquet('f5aux.parquet')

In [17]:
#Filtro el dataframe por recommend = 0 y sentiment = 0
df_filtered = df4[(df4["recommend"] == 0) & (df4["sentiment"] == 0)]

#Elijo el año que me interesa
df_year = df_filtered[df_filtered["posted_year"] == 2013]

#Calculo el top 3 de developer 
df_top3 = df_year.groupby("developer")["developer"].count().nlargest(3)



In [18]:
#Muestro resultados con la visualización solicitada
df_dict = df_top3.to_dict()
df_list = [{f"Puesto {i+1}" : v} for i, v in enumerate(df_dict)]
df_list

[{'Puesto 1': 'Infinity Ward'},
 {'Puesto 2': 'Valve'},
 {'Puesto 3': 'Avalanche Studios'}]